In [1]:
import pandas as pd
import numpy
from sklearn import model_selection, preprocessing

In [2]:
df = pd.read_csv('Unemployment_mental_illness_survey.csv')
# df.head()

In [3]:
# Original column names are combersome. Change to convenient ones.

clean_df=df.rename(columns={"I am currently employed at least part-time":"employed",
                   "I identify as having a mental illness":"mental_illness",
                   "I have my own computer separate from a smart phone":"own_computer",
                   "I have been hospitalized before for my mental illness":"hospitalized",
                   "How many days were you hospitalized for your mental illness":"hospitalized_days",
                   "I am legally disabled":"disabled","I have my regular access to the internet":"internet",
                   "I live with my parents":"with_parents","I have a gap in my resume":"gap",
                   "Total length of any gaps in my resume in\xa0months.":"months_gap",
                   "Annual income (including any social welfare programs) in USD":"income_with_social",
                   "I am unemployed":"unemployed",
                   "I read outside of work and school":"read",
                   "Annual income from social welfare programs":"welfare_income",
                   "I receive food stamps":"food_stamp",
                   "I am on section 8 housing":"section_8",
                   "How many times were you hospitalized for your mental illness":"times_hospitalized",
                   "Lack of concentration":"Lack_concentration",
                   "Obsessive thinking":"Obsessive_thinking",
                   "Mood swings":"Mood_swings",
                   "Panic attacks":"Panic_attacks",
                   "Compulsive behavior":"Compulsive_behavior",
                   "Household Income":"Household_Income",
                   "Device Type":"Device"
                  })
clean_df.head()

,employed,mental_illness,Education,own_computer,hospitalized,hospitalized_days,disabled,internet,with_parents,gap,...,Obsessive_thinking,Mood_swings,Panic_attacks,Compulsive_behavior,Tiredness,Age,Gender,Household_Income,Region,Device
0,0,0,High School or GED,0,0,0.0,0,1,0,1,...,1.0,0.0,1.0,0.0,0.0,30-44,Male,"$25,000-$49,999",Mountain,Android Phone / Tablet
1,1,1,Some Phd,1,0,0.0,0,1,0,0,...,0.0,0.0,1.0,0.0,1.0,18-29,Male,"$50,000-$74,999",East South Central,MacOS Desktop / Laptop
2,1,0,Completed Undergraduate,1,0,0.0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,30-44,Male,"$150,000-$174,999",Pacific,MacOS Desktop / Laptop
3,0,0,Some Undergraduate,1,0,NaN,0,1,1,1,...,0.0,0.0,0.0,0.0,0.0,30-44,Male,"$25,000-$49,999",New England,Windows Desktop / Laptop
4,1,1,Completed Undergraduate,1,1,35.0,1,1,0,1,...,1.0,1.0,1.0,1.0,1.0,30-44,Male,"$25,000-$49,999",East North Central,iOS Phone / Tablet


In [4]:
# Eliminate uninformative data

del clean_df['unemployed']
del clean_df['Device']
del clean_df['income_with_social']
clean_df = clean_df[clean_df.Household_Income != 'Prefer not to answer']
clean_df.head()

,employed,mental_illness,Education,own_computer,hospitalized,hospitalized_days,disabled,internet,with_parents,gap,...,Depression,Obsessive_thinking,Mood_swings,Panic_attacks,Compulsive_behavior,Tiredness,Age,Gender,Household_Income,Region
0,0,0,High School or GED,0,0,0.0,0,1,0,1,...,1,1.0,0.0,1.0,0.0,0.0,30-44,Male,"$25,000-$49,999",Mountain
1,1,1,Some Phd,1,0,0.0,0,1,0,0,...,1,0.0,0.0,1.0,0.0,1.0,18-29,Male,"$50,000-$74,999",East South Central
2,1,0,Completed Undergraduate,1,0,0.0,0,1,0,0,...,0,0.0,0.0,0.0,0.0,0.0,30-44,Male,"$150,000-$174,999",Pacific
3,0,0,Some Undergraduate,1,0,NaN,0,1,1,1,...,0,0.0,0.0,0.0,0.0,0.0,30-44,Male,"$25,000-$49,999",New England
4,1,1,Completed Undergraduate,1,1,35.0,1,1,0,1,...,1,1.0,1.0,1.0,1.0,1.0,30-44,Male,"$25,000-$49,999",East North Central


In [5]:
# Change strings to categories

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

clean_df = pd.get_dummies(clean_df, columns=['Education', 'Age', 'Gender', 'Region'])
clean_df = clean_df.fillna(0)


Using TensorFlow backend.


In [6]:
# Set household income as the prediction compared to other factors

X = clean_df.drop(columns="Household_Income")
y = clean_df["Household_Income"].values.reshape(-1, 1)
print(X.shape, y.shape)

(296, 46) (296, 1)


In [7]:
# Split training and test data

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state = 18)

In [8]:
# This is a classification problem, so let us try a random forest

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

In [9]:
# Change string columns to one-hot categories

labelencoder = LabelEncoder()
labelencoder.fit(y_train)
y_train_encoded = labelencoder.transform(y_train)
y_test_encoded = labelencoder.transform(y_test)
y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)

C:\Users\jason\Anaconda3\envs\JasonsWorld\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\jason\Anaconda3\envs\JasonsWorld\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
# Fit the model to the training data and calculate the scores for the training and testing data

model.fit(x_train, y_train_categorical)

training_score = model.score(x_train, y_train_categorical)
testing_score = model.score(x_test, y_test_categorical)

print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

Training Score: 0.990990990990991
Testing Score: 0.02702702702702703


In [12]:
# Let us also try deep learning
# Next six cells, code provided by tutor

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

In [13]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(x_train)
X_train_scaled = X_scaler.transform(x_train)
X_test_scaled = X_scaler.transform(x_test)

In [17]:
model2 = Sequential()
model2.add(Dense(units=100, activation='relu', input_dim=46))
model2.add(Dense(units=10, activation='softmax'))

In [18]:
# Compile and fit the model
model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
# set early stopping as callback
callbacks = [EarlyStopping(monitor='val_loss', patience=2)]
model2.fit(
    X_train_scaled,
    y_train_categorical,
    callbacks=callbacks,
    epochs=50,
    shuffle=True,
    verbose=2
)

Epoch 1/50
 - 1s - loss: 2.5255 - accuracy: 0.0135
Epoch 2/50
 - 0s - loss: 2.3486 - accuracy: 0.0541
Epoch 3/50
 - 0s - loss: 2.2178 - accuracy: 0.1802
Epoch 4/50
 - 0s - loss: 2.1322 - accuracy: 0.2523
Epoch 5/50
 - 0s - loss: 2.0697 - accuracy: 0.2568
Epoch 6/50
 - 0s - loss: 2.0209 - accuracy: 0.2613
Epoch 7/50
 - 0s - loss: 1.9819 - accuracy: 0.2928
Epoch 8/50
 - 0s - loss: 1.9466 - accuracy: 0.3063
Epoch 9/50
 - 0s - loss: 1.9157 - accuracy: 0.3243
Epoch 10/50
 - 0s - loss: 1.8874 - accuracy: 0.3288
Epoch 11/50
 - 0s - loss: 1.8575 - accuracy: 0.3514


C:\Users\jason\Anaconda3\envs\JasonsWorld\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 12/50
 - 0s - loss: 1.8288 - accuracy: 0.3649
Epoch 13/50
 - 0s - loss: 1.8017 - accuracy: 0.3694
Epoch 14/50
 - 0s - loss: 1.7784 - accuracy: 0.3829
Epoch 15/50
 - 0s - loss: 1.7519 - accuracy: 0.4144
Epoch 16/50
 - 0s - loss: 1.7279 - accuracy: 0.4279
Epoch 17/50
 - 0s - loss: 1.7019 - accuracy: 0.4414
Epoch 18/50
 - 0s - loss: 1.6803 - accuracy: 0.4414
Epoch 19/50
 - 0s - loss: 1.6562 - accuracy: 0.4550
Epoch 20/50
 - 0s - loss: 1.6362 - accuracy: 0.4730
Epoch 21/50
 - 0s - loss: 1.6128 - accuracy: 0.5000
Epoch 22/50
 - 0s - loss: 1.5912 - accuracy: 0.5090
Epoch 23/50
 - 0s - loss: 1.5715 - accuracy: 0.4865
Epoch 24/50
 - 0s - loss: 1.5497 - accuracy: 0.5135
Epoch 25/50
 - 0s - loss: 1.5304 - accuracy: 0.5180
Epoch 26/50
 - 0s - loss: 1.5100 - accuracy: 0.5360
Epoch 27/50
 - 0s - loss: 1.4916 - accuracy: 0.5405
Epoch 28/50
 - 0s - loss: 1.4733 - accuracy: 0.5405
Epoch 29/50
 - 0s - loss: 1.4540 - accuracy: 0.5450
Epoch 30/50
 - 0s - loss: 1.4365 - accuracy: 0.5360
Epoch 31/50


In [20]:
model_loss, model_accuracy = model2.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 2.2879571785797945, Accuracy: 0.1756756752729416
